In [ ]:
import numpy
import pandas
from keras.models import Sequential
from keras import layers
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras.optimizers import Adam

In [ ]:
seed = 7
numpy.random.seed(seed)


### Loading & splitting data 

In [ ]:
dataframe = pandas.read_csv("iris.csv", header=None)
dataset = dataframe.values


In [ ]:
X = dataset[:,0:4].astype(float)
Y = dataset[:,4]


### Converting Labels

In [ ]:
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
dummy_y = np_utils.to_categorical(encoded_Y)

In [ ]:
dummy_y.shape

### Creating baseline model 

In [ ]:
def create_baseline():
    model=Sequential()
    model.add(Dense(8, activation='relu',input_shape=(4,)))
    model.add(Dense(3,activation='softmax'))
    model.compile(optimizer=Adam(lr=0.002), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
estimator = KerasClassifier(build_fn=create_baseline, epochs=200, batch_size=5, verbose=0)

In [ ]:
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

In [ ]:
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


### Smaller model 

In [ ]:
def create_smaller():
    model=Sequential()
    model.add(Dense(4, activation='relu',input_shape=(4,)))
    model.add(Dense(3,activation='softmax'))
    model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
estimator = KerasClassifier(build_fn=create_smaller, epochs=100, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


### Larger model 

In [ ]:
def create_larger():
    model=Sequential()
    model.add(Dense(32, activation='relu',input_shape=(4,)))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(4, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
estimator = KerasClassifier(build_fn=create_larger, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=20, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

### Model that overfits 

In [ ]:
def create_overfitting_model():
    model=Sequential()
    model.add(Dense(32, activation='relu',input_shape=(4,)))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    model.compile(optimizer=Adam(lr=0.08), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
estimator = KerasClassifier(build_fn=create_overfitting_model, epochs=400, batch_size=10, verbose=0)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("overfitting_Result: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

### Model with keras functional API 

In [ ]:
import keras

def functional_api():
    inputs=keras.Input(shape=(4,))
    hl_1=layers.Dense(4,activation='relu')(inputs)
    hl_2=layers.Dense(16, activation='relu')(hl_1)
    outputs=layers.Dense(3, activation='softmax')(hl_2)
    model=keras.Model(inputs,outputs)
    model.compile(optimizer=Adam(lr=0.002),loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
estimator = KerasClassifier(build_fn=functional_api, epochs=100, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("functional_API_Result: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [ ]:
import keras
input=keras.Input(shape=(4,))
class MyModel(keras.Model):
    def __init__(self):
        super(MyModel,self).__init__()
        self.dense1=layers.Dense(64,activation='relu')
        self.dense2=layers.Dense(32,activation='relu')
        self.dense3=layers.Dense(16,activation='relu')
        self.dense4=layers.Dense(3,activation='softmax')
    def call(self,inputs):
        x=self.dense1(inputs)
        x=self.dense2(x)
        x=self.dense3(x)
        return self.dense4(x)
model=MyModel()
model.compile(optimizer=Adam(lr=0.002),loss='binary_crossentropy',metrics=['accuracy'])
model.fit(X,encoded_Y,epochs=50,batch_size=3)

### Applying kfold cross-validation on model without scikit-learn 

In [ ]:
k = 4
num_val_samples = len(X) // k
num_epochs = 100
all_scores = []

In [ ]:
for i in range(k):
    print('processing fold #', i)
    val_data = X[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = dummy_y[i * num_val_samples: (i + 1) * num_val_samples]
    partial_train_data = numpy.concatenate([X[:i * num_val_samples],X[(i + 1) * num_val_samples:]],axis=0)
    partial_train_targets = numpy.concatenate([dummy_y[:i * num_val_samples],dummy_y[(i + 1) * num_val_samples:]],axis=0)
model = create_baseline()
model.fit(partial_train_data, partial_train_targets,epochs=num_epochs, batch_size=1, verbose=0)
val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
all_scores.append(val_mae)

In [ ]:
all_scores